In [2]:
config = {
        # directories
        "project_name": "Test",
        "metadata_dir": '/projects/home/mmasood1/trained_model_predictions/Tox21/Frozen_BERT/uniform_sampling/',
        "target_file": "/projects/home/mmasood1/arslan_data_repository/Tox21/complete_Tox21.csv",
        "BERT_features_file":"/projects/home/mmasood1/arslan_data_repository/Tox21/Tox21_BERT_features.csv",
        "model_weights_dir" : '/projects/home/mmasood1/Model_weights/Tox21/',
        "pos_weights": "/projects/home/mmasood1/arslan_data_repository/Tox21/pos_weights.csv",
        "class_weights": "/projects/home/mmasood1/arslan_data_repository/Tox21/target_weights.csv",
        
        # data
        "features_type" :"BERT",
        "FP_size" : 1024,
        "train_frac": 0.8,

        # architechture
        "input_dim": 768,
        "hidden_dim": 128,
        "depth" : 1,
        "dropout_p": 0.2,
        "BatchNorm1d": True,
        "use_skip_connection": True,
    
        # training
        "optim": 'Adam',#SGD
        "lr_schedulers": "CosineAnnealingLR",
        "lr": 1e-3,
        "l2_lambda": 0.0,
        "optm_l2_lambda": 1e-3,
        "epochs": 20,
        "compute_metric_after_n_epochs": 5,
        "batch_size": 64,
        "EarlyStopping": False, 
        "pretrained_model": False,
        
        # loss
        "missing" : 'nan',
        "alpha": 0.0,
        "beta": 0.0,
        "gamma":0.0,

        "gpu": [0],
        "accelerator": "gpu",
        "device" :torch.device("cuda"),
        "return_trainer": True, 
        "save_predicitons" : True,
        "Final_model": False,

        # active learning
        "num_forward_passes": 5,
        "num_itterations": 3,
        "sampling_strategy": "uniform",
        "n_query":500
    }

# get targets information
data = pd.read_csv(config["target_file"])
target_names = data.loc[:, "NR-AR":"SR-p53"].columns.tolist()

config["num_of_tasks"] = len(target_names)
config["selected_tasks"] = target_names

config["loss_type"] = "Focal_loss" #"Focal_loss",# "BCE","Focal_loss_v2"
config["seed"] = 42

In [ ]:
# Splitting by using deepchem
train_set, test_set = scafoldsplit_train_test(config)

randomstratifiedsplitter = dc.splits.RandomStratifiedSplitter()
pool_set, val_set, initial_set = randomstratifiedsplitter.train_valid_test_split(train_set,
                                                                                 frac_train=0.85,
                                                                                 frac_valid=0.134,
                                                                                 frac_test=0.016,
                                                                                 seed=42)

print("train_set", sorted(np.nansum(train_set.y, axis=0)))
print("test_set", sorted(np.nansum(test_set.y, axis=0)))
print("pool_set", sorted(np.nansum(pool_set.y, axis=0)))
print("val_set", sorted(np.nansum(val_set.y, axis=0)))
print("initial_set", sorted(np.nansum(initial_set.y, axis=0)))
train_set.y.shape, test_set.y.shape, pool_set.y.shape, val_set.y.shape, initial_set.y.shape


train_test_features (6121, 768) (1531, 768)
train_test_targets (6121, 12) (1531, 12)
train_set [114.0, 183.0, 185.0, 203.0, 249.0, 249.0, 256.0, 280.0, 587.0, 640.0, 687.0, 687.0]
test_set [42.0, 47.0, 52.0, 55.0, 63.0, 88.0, 91.0, 139.0, 140.0, 177.0, 204.0, 218.0]
pool_set [99.0, 149.0, 153.0, 169.0, 212.0, 213.0, 219.0, 235.0, 500.0, 534.0, 585.0, 587.0]
val_set [13.0, 25.0, 29.0, 30.0, 30.0, 33.0, 37.0, 40.0, 79.0, 91.0, 93.0, 94.0]
initial_set [2.0, 3.0, 3.0, 4.0, 4.0, 5.0, 5.0, 7.0, 8.0, 8.0, 9.0, 13.0]


((6121, 12), (1531, 12), (5203, 12), (820, 12), (98, 12))

In [ ]:
# Who cares about deepchem data_object, trash it
initial_set = convert_to_dataframe(initial_set, config["selected_tasks"])
val_set = convert_to_dataframe(val_set, config["selected_tasks"])
pool_set = convert_to_dataframe(pool_set, config["selected_tasks"])
test_set = convert_to_dataframe(test_set, config["selected_tasks"])

In [5]:
for itteration in range(config["num_itterations"]):

    from utils.models import Vanilla_MLP_classifier
    seed_everything(seed = config["seed"])
    config["itteration"] = itteration
    config["model_name"] = rf'itteration_{config["itteration"]}_s{config["seed"]}_alpha_{config["alpha"]}_gamma_{config["gamma"]}_loss_type_{config["loss_type"]}_λ{config["optm_l2_lambda"]}'
    
    # get dataloaders
    train_dataloader = convert_dataframe_to_dataloader(dataframe= initial_set, config = config, shuffle= True)
    val_dataloader = convert_dataframe_to_dataloader(dataframe= val_set, config = config, shuffle= False)
    test_dataloader = convert_dataframe_to_dataloader(dataframe= test_set, config = config, shuffle= False)
    pool_dataloader = convert_dataframe_to_dataloader(dataframe= pool_set, config = config, shuffle= False)

    # Train model
    config["training_steps"] = len(train_dataloader)
    trained_model, run, trainer = wandb_init_model(model = Vanilla_MLP_classifier, 
                                                            train_dataloader = train_dataloader,
                                                            val_dataloader =val_dataloader,
                                                            config = config, 
                                                            model_type = 'MLP')
    
    wandb.finish()

    ###### Model Evaluation #############
    # make dir
    query_set_dir = config["metadata_dir"] + "query_set/"
    result_dir = config["metadata_dir"]
    os.makedirs(query_set_dir, exist_ok = True)
    os.makedirs(result_dir, exist_ok = True)

    # Evaluation
    trained_model = trained_model.eval()
    targets, pred_mean, pred_var, all_pred = get_pred_with_uncertainities(test_dataloader, trained_model,
                                                        n_samples=test_set.shape[0],
                                                        n_classes=config["num_of_tasks"],
                                                        cal_uncert=False,
                                                        num_forward_passes= 1)
    metrics = compute_binary_classification_metrics_MT(targets, pred_mean, missing='nan')

    metrics = metrics.append(metrics.mean(), ignore_index=True)
    metrics.insert(0, 'Tasks', config["selected_tasks"] + ['mean'])
    metrics.to_csv(result_dir + f'metrics_{config["model_name"]}.csv', index=False)
    print(metrics.mean())

    query_set, updated_training_set, updated_poolset = active_learning_loop(trained_model,
                                                                        pool_dataloader, 
                                                                        initial_set,
                                                                        pool_set, 
                                                                        config)
    query_set.to_csv(query_set_dir + f'query_set_{config["model_name"]}.csv', index=False)
    del initial_set,pool_set
    del train_dataloader,val_dataloader,test_dataloader,pool_dataloader, trained_model

    initial_set = updated_training_set.copy()
    pool_set = updated_poolset.copy()

    torch.cuda.empty_cache()
    gc.collect()

    print('++++++++++++++++++++++++++++++++++++++++++++++++++++')
    gpu_memory_status = torch.cuda.memory_allocated() / (1024 ** 3)
    print("GPU Memory Status (after clearing):", gpu_memory_status)
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++')

Seed set to 42


After merging (98, 768)
After merging (820, 768)
After merging (1531, 768)
After merging (5203, 768)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:634: Checkpoint directory /projects/home/mmasood1/Model_weights/Tox21/ exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type              | Params
-------------------------------------------------------------
0 | weighted_creterien     | BCEWithLogitsLoss | 0     
1 | non_weighted_creterian | BCEWithLogitsLoss | 0     
2 | FL                     | FocalLoss         | 0     
3 | input_laye

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
/tmp/ipykernel_9522/65254225.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append(metrics.mean(), ignore_index=True)
/tmp/ipykernel_9522/65254225.py:43: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(metrics.mean())


balanced_acc         0.643913
f1_score             0.309555
specificity          0.714908
sensitivity          0.572919
roc_auc              0.673822
AUPR                 0.230266
average_precision    0.234900
ECE                  0.051280
ACE                  0.053322
dtype: float64
initial_counts 1994 initial_poolset_count 51950 query_counts 500 updated_training_counts 2494 updated_poolset_counts 51450
++++++++++++++++++++++++++++++++++++++++++++++++++++
GPU Memory Status (after clearing): 0.00044155120849609375
++++++++++++++++++++++++++++++++++++++++++++++++++++
